# ライフゲーム

pros:
- Jupyter, Colab 両方で動く。

cons:
- Binderで動かない(?)
- アニメーション化が遅い。
- アニメーションをノートブックに保存するとサイズが大きくなる。

In [ ]:
%%capture
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import random
import numpy as np
import numba

L = 100
FIG_SIZE = 8
MAX_T = 1000


def init_cells(a):
    for i in range(L):
        for j in range(L):
            if random.random() < 0.3:
                a[i, j] = True


@numba.njit
def update_cells(a, b):
    for i in range(L):
        for j in range(L):
            left = i - 1
            right = i + 1
            up = j - 1
            down = j + 1
            
            if left < 0:
                left += L
            if right >= L:
                right -= L
            if up < 0:
                up += L
            if down >= L:
                down -= L
    
            count = 0
            if b[left, up]:
                count += 1
            if b[i, up]:
                count += 1
            if b[right, up]:
                count += 1
            if b[left, j]:
                count += 1
            if b[right, j]:
                count += 1
            if b[left, down]:
                count += 1
            if b[i, down]:
                count += 1
            if b[right, down]:
                count += 1
                
            if count == 2:
                a[i, j] = b[i, j]
            elif count == 3:
                a[i, j] = True
            else:
                a[i, j] = False


a = np.zeros(shape=(L, L), dtype=bool)
b = a.copy()
t = 0

init_cells(a)

fig, ax = plt.subplots(figsize=(FIG_SIZE, FIG_SIZE))
ax.set_axis_off()
tlabel = ax.text(L, 0, f't = {t}')
image = ax.matshow(a.T, vmin=0, vmax=1)

def update(frame):
    global a, b, t
    if frame <= t:
        return
    update_cells(b, a)
    a, b = b, a
    t += 1
    tlabel.set_text(f't = {t}')
    image.set_data(a.T)

anim = FuncAnimation(fig, update, frames=MAX_T + 1)

In [ ]:
HTML(anim.to_jshtml())